# Xavier Obezo - Protect Earth

The goal of this project is to use your Astro 300 python programming skills to answer the 3 questions below.

Your aim is to:

- Create a well commmented python notebook that shows your programming.
- Keep to the class style guide (good variable names).
- Do not hard code any common physical constants.
- Easy to read and neat output that **clearly** shows the answers to the questions.
- There should be no calculations outside of the `class` definition.

The starting point is the dataset **`./Data/PHA.csv`** that contains data for 10 objects classified as potentially hazardous asteroids.

## Due between Fri Feb 8 (5pm) - Tue Feb 12 (Noon)
 * `Submit < 5pm Feb 8` - **30 pts** possible.
 * `5pm Feb 8 < Submit < 5pm Feb 10` - **20** pts possible.
 * `5pm Feb 10 < Submit < Noon Feb 12` - **10** pts possible.
 * `File -> Download as... -> HTML (right-click to save HTML file)`
 * `Upload HTML file to Canvas`
 * `Make sure to change the filename to your name!`
 * `Make sure to change the Title to your name!`

In [1]:
import numpy as np
import pandas as pd

from astropy import units as u
from astropy import constants as const

In [2]:
#I don't really know where the best place to define these would be
ton_TNT = u.def_unit("Tons of TNT", 4.18e9*u.J)
gigaton_TNT =u.def_unit("Gigatons of TNT", 1e9 *ton_TNT)

class SpaceRock(object):
    
    def __init__(self, name = None, ab_mag = None, albedo = None, 
                 semi_major= None, ecc = None,
                 density = 3000 *u.kg/(u.m*u.m*u.m)):
        """
        Parameters
        ----------
        name : string
            Name of the target
        ab_mag : array-like
            Absolute Magnitude of each Asteroid
        albedo : array-like
            Albedo of each Asteroid
        semi_major : array-like
            Semi Major Axis of each Asteroid in AU
        ecc : array-like
            Eccentricity of each Asteroid
        """
        self.name = name
        self.ab_mag = ab_mag
        self.albedo = albedo
        self.semi_major = semi_major * u.au
        self.ecc = ecc
        self.density = density
    
    def diameter(self):
        """
        Determine the diameter (in km) of the Asteroids
        """
        result = (1329.0 / np.sqrt(self.albedo)) * (10 ** (-0.2 * self.ab_mag))
        return result * u.km
    
    def orbital_velocity(self, distance):
        """
        Determines the velocity at a given distance
        """
        # v= sqrt(GM * (2/r - 1/a) )
        velocity = np.sqrt(const.G*const.M_sun *
                           (2/distance - 1/self.semi_major) )
        return velocity.decompose()
    
    def collision_velocity(self):
        """
        Determines the velocity if it were to collide with Earth
        """
        V_earth = 30 * u.km/u.s
        V_encounter = self.orbital_velocity(1*u.au) - V_earth
        
        V_escape = 11.2 * u.km/u.s
        V_impact = np.sqrt(V_escape**2 + V_encounter**2)
        
        return V_impact.decompose()
    
    def mass(self):
        """
        Finds the mass of each Asteriod
        """
        
        mass = self.density * (1/6)*np.pi*self.diameter()**3
        return mass.decompose()
    
    def kinetic_energy(self):
        """
        Finds the kinetic energy in joules
        """
        #KE = 1/2 * mv^2
        KE = 1/2 * self.mass() * self.collision_velocity()**2
        
        return KE.to(u.joule)
    
    def kinetic_energy_TNT(self):
        """
        Finds the kinetic energy in terms of gigatons of TNT
        """
        return self.kinetic_energy().to(gigaton_TNT)
    
    def potential_energy(self):
        """
        Finds energy needed to destroy asteriod in joules
        """
        #6/5* GM^2/D
        PE = 6/5 * const.G*self.mass()**2/self.diameter()
        return PE.to(u.joule)
    
    def potential_energy_TNT(self):
        """
        Finds potential energy in terms of tons of TNT
        """
        return self.potential_energy().to(ton_TNT)
    
    def crater_size(self):
        """
        Finds the size of a crater hitting Earth from the asteriod
        """
        #Couldn't find Earths density on the constants page so here's from google
        targDensity = 5.51 * u.g/u.cm**3
        size = 1.8*self.density**0.11 * targDensity**-0.33 * const.g0**-0.22 *self.diameter()**0.13 * self.kinetic_energy()**0.22
        # Wierd issue: getting unit as m^1 which I guess is not equal to u.m
        # To fix this I'm going to force the unit. But I don't like this solution
        # I know the decompose will give units in meters for this case. If I were to want to push this code
        #   I would want to fix this. But I don't know how else to.
        #   I think this issue is due to approximation of the unit. It isn't 1 but really close.
        size = size.decompose().value * u.m
        size = size.to(u.km)
        return size

# Proof that it really is in meters but it won't let me change it's type (for crater_size)
# In order for these commented out parts to work you would need to comment out the forced unit parts of the code
#asteriods.crater_size()[0].decompose() * u.m**1
# This code below will throw an error




## Read in the dataset `./Data/PHA.csv`

In [3]:
table = pd.read_csv('./Data/PHA.csv')

In [11]:
#name = None, ab_mag = None, albedo = None, semi_major= None, ecc = None
new = pd.read_csv('Rocks.csv')

## ...and use the data to call `SpaceRock`

In [13]:


names = new['Name'].values
ab_mags = new['H'].values
albedos = new['A'].values
eccentricities = table['ecc'].values
semi_majors = table['a'].values
asteriods = SpaceRock(name = names, ab_mag = ab_mags,
                      albedo = albedos, semi_major= semi_majors,
                      ecc = eccentricities)
asteriods.diameter()

<Quantity [120.89235138, 212.19484847,  98.29231689, 255.93228029,
            80.73576608, 113.48183465, 103.06138779, 105.61755433,
           108.33644849, 115.88593881, 156.55239169, 107.59617502,
           173.9842008 , 100.17817157,  65.86424222,  70.63309154,
           214.57713626, 124.14845008, 126.97222292, 221.78526349,
           149.80896907,  99.81386504, 147.82605694, 302.38556942,
           115.37978559, 165.72136026,  69.82732661, 113.25787015,
           112.56588061,  93.4460744 , 164.73577588,  60.19161543,
            82.02714378,  95.40545153, 103.12723872,  98.07880252,
            71.78451052,  58.09914639, 122.53834916, 138.08642121,
           122.14426895,  83.41280316,  85.86109506,  46.52897064,
           118.65508256,  55.9015188 ,  69.23787572, 120.53762508,
            66.45691237,  78.37435821] km>

---

## There should **only** be calls to the `SpaceRock` class below this line (and formatting)

---

## 1 - Determine the speed of each of the PHAs at **r** = 1 AU.


* Make sure you use units.
* Express your answer SI units with 2 digits after the decimal.
* The output should be a series of lines like:
  * `The speed of NAME at 1 AU is VALUE UNIT.` 

In [6]:
names1 = asteriods.name
for idx,value in enumerate(asteriods.orbital_velocity(1*u.au)):
    print("The speed of {0:s} at 1 AU is {1:.2f}."
          .format(names1[idx],value.to(u.km/u.s)))


The speed of Icarus at 1 AU is 30.87 km / s.
The speed of Geographos at 1 AU is 32.63 km / s.
The speed of Apollo at 1 AU is 34.22 km / s.
The speed of Midas at 1 AU is 35.72 km / s.
The speed of Adonis at 1 AU is 36.05 km / s.
The speed of Phaethon at 1 AU is 32.80 km / s.
The speed of Dionysus at 1 AU is 37.03 km / s.
The speed of Wilson-Harrington at 1 AU is 37.92 km / s.
The speed of Vishnu at 1 AU is 30.62 km / s.
The speed of Toutatis at 1 AU is 37.73 km / s.


## 2 - Determine the kinetic energy each PHA would have if they impacted the surface of the Earth

- Assume asteroids have $\rho$ = 3,000 kg/m$^3$
- Express your answer in gigatons of TNT with 1 digit after the decimal
- 1 ton TNT $= 4.18 \times 10^9$ J
- 1 gigaton = 1e9 tons

* The output should be a series of lines like:
  * `The kinetic energy of NAME hitting the Earth is VALUE UNIT.` 

In [7]:
names2 = asteriods.name
for idx,value in enumerate(asteriods.kinetic_energy_TNT()):
    print("The kinetic energy of {0:s} hitting the Earth is {1:.1f}."
          .format(names2[idx],value))


The kinetic energy of Icarus hitting the Earth is 11.1 Gigatons of TNT.
The kinetic energy of Geographos hitting the Earth is 134.4 Gigatons of TNT.
The kinetic energy of Apollo hitting the Earth is 89.9 Gigatons of TNT.
The kinetic energy of Midas hitting the Earth is 423.3 Gigatons of TNT.
The kinetic energy of Adonis hitting the Earth is 3.0 Gigatons of TNT.
The kinetic energy of Phaethon hitting the Earth is 2799.8 Gigatons of TNT.
The kinetic energy of Dionysus hitting the Earth is 174.1 Gigatons of TNT.
The kinetic energy of Wilson-Harrington hitting the Earth is 1891.1 Gigatons of TNT.
The kinetic energy of Vishnu hitting the Earth is 1.3 Gigatons of TNT.
The kinetic energy of Toutatis hitting the Earth is 431.7 Gigatons of TNT.


## 3 - Determine how many nuclear weapons, with a yield of 1 ton-TNT, will be needed to destroy each of the PHAs.

- Express your answer in the number of 1 ton-TNT weapons with 1 digit after the decimal
- 1 ton TNT $= 4.18 \times 10^9$ J

* The output should be a series of lines like:
  * `It would take VALUE 1 ton nuclear weaponds to destroy NAME.` 

In [8]:
names3 = asteriods.name
for idx,value in enumerate(asteriods.potential_energy_TNT()):
    print("It would take {1:.1f} 1 ton nuclear weapons to destroy {0:s}."
          .format(names3[idx],value.value))


It would take 13.3 1 ton nuclear weapons to destroy Icarus.
It would take 787.0 1 ton nuclear weapons to destroy Geographos.
It would take 352.7 1 ton nuclear weapons to destroy Apollo.
It would take 3957.4 1 ton nuclear weapons to destroy Midas.
It would take 1.0 1 ton nuclear weapons to destroy Adonis.
It would take 122681.6 1 ton nuclear weapons to destroy Phaethon.
It would take 762.0 1 ton nuclear weapons to destroy Dionysus.
It would take 35878.6 1 ton nuclear weapons to destroy Wilson-Harrington.
It would take 0.4 1 ton nuclear weapons to destroy Vishnu.
It would take 3143.5 1 ton nuclear weapons to destroy Toutatis.


----

# <font color=blue>Ravenclaw</font>

## 4 - Determine the size of the crater each PHA would make if they impacted the surface of the Earth

- Assume asteroids and the Earth's surface have a density = 3,000 kg/m$^3$
* The output should be a series of lines like:
  * `If NAME hit the Earth, it would form a crater with a size of VALUE UNIT.`

In [9]:
names4 = asteriods.name
for idx,value in enumerate(asteriods.crater_size()):
    print("If {0:s} hit the Earth, it would form a crater with a size of {1:.1f}."
          .format(names4[idx],value))

If Icarus hit the Earth, it would form a crater with a size of 7.7 km.
If Geographos hit the Earth, it would form a crater with a size of 14.8 km.
If Apollo hit the Earth, it would form a crater with a size of 13.3 km.
If Midas hit the Earth, it would form a crater with a size of 19.9 km.
If Adonis hit the Earth, it would form a crater with a size of 5.4 km.
If Phaethon hit the Earth, it would form a crater with a size of 33.0 km.
If Dionysus hit the Earth, it would form a crater with a size of 15.7 km.
If Wilson-Harrington hit the Earth, it would form a crater with a size of 29.3 km.
If Vishnu hit the Earth, it would form a crater with a size of 4.4 km.
If Toutatis hit the Earth, it would form a crater with a size of 19.9 km.


---

## Some Orbital Mechanics

Kepler's first law says: *The orbit of every planet is an ellipse with the sun at one focus*. The Semimajor axis **a** and the eccentricity **ecc** parametrize the size and shape of the ellipse. The units of **a** in our dataset are Astronomical Units (AU), the average distance between the Sun and the Earth.

![Orbit Diagram](images/Orbit.jpg)

For a closed elliptical orbit (orbits gravitationally bound to the Sun), $ecc = \sqrt{1 - {b^2}/{a^2}}$, where **a** and **b** are the semimajor and semiminor axes. As you can see from the equation, when **a** = **b**, **ecc** = 0 (a circle), and when **a** $>>$ **b**, **ecc** approaches 1. When **ecc** = 1, the orbit is a parabolic orbit (just bound). When **ecc** $>$ 1 the orbit is a hyperbolic orbit (unbound).

---

The speed of an object on an elliptical orbit around the Sun at a distance **r** from the Sun is:

$$ \large
v\ =\ \sqrt{GM_{\odot}\ \left(\frac{2}{r} - \frac{1}{a}\right)}
$$

Watch your units! Both `r` and `a` should have the same units

---

## Encountering the Earth

The encounter speed of an asteroid meeting the Earth at 1 AU is (assuimg aligned prograde orbits):

$$ \large
V_{\textrm{encounter}}\ =\ V_{\textrm{asteroid at 1AU}}\ -\ V_{\textrm{Earth}}
$$

Where $V_{\textrm{Earth}}\ =\ 30\ \textrm{km/s}$

## Hitting the Earth

The impact speed of an asteroid hitting the Earth is:

$$ \large
V_{\textrm{impact}}\ =\ \sqrt{V_{\textrm{encounter}}^2 + V_{\textrm{escape}}^2}
$$

Where $V_{\textrm{escape}}\ = 11.2\ \textrm{km/s}$

---

## Blowing up an asteroid

The self gravitational potential energy of a uniform sphere of mass (M) and diameter (D) is:

$$ \large
PE \ = \ \frac{6}{5} \cdot \frac{GM^2}{D}
$$

This is the amount of energy you need to give the sphere to move all of its components pieces infinitely far away (i.e. to destroy it!).

Remember that the mass and diameter of the asteroid is derived from its absolute magnitude **H** and albedo **A**, and density.

---

## Size of Crater

This size of the crater formed by an impacting asteroid can be estimated by:

$$ \large
\textrm{Size of Crater} = 1.8\ \rho_{p}^{0.11}\ \rho_{t}^{-0.33}\ g^{-0.22}\ D^{0.13}\ W^{0.22}
$$

Where

* $\rho_{p}$ is the asteroid density
* $\rho_{t}$ is the target density
* $D$ is the asteroid diameter
* $W$ is the kinetic energy of the asteroid hitting the Earth's surface

### Important Note!!!!

This type of equation is sometimes refered to as an *emperical scaling relation*

- The units do not work out as written
- You need to put all variables into the same unit system (such as mks)
- Then use .value to just pull off the numerical value
- The answer will be in whatever unit system you used (such as mks)

- Remember: Do all of this work as a `method` in the `SpaceRock()` class!
